##### jz3702

In [14]:
import torch
import numpy as np  
import tensorflow as tf
import re
import os
import glob
import datetime

## 1. Analysis on Model Performance (Word Error Rate)


### 1.1 Get the epoch number with best validation loss

In [32]:
directory='./exp/'

model_epoch_info = {}

for subdir in os.listdir(directory):
    model_checkpoint_epoch = 1
    for file in os.listdir(directory + subdir):
        if re.match(r'epoch.*\.pt$', file):
            if int(file[6:-3]) > model_checkpoint_epoch:
                model_checkpoint = file
                model_checkpoint_epoch = int(file[6:-3])

    print(subdir)
    model = torch.load(os.path.join(directory, subdir, model_checkpoint), map_location=torch.device('cpu'))
    # print(model)
    print('Best Validation Loss Epoch: ', model['best_valid_epoch'])
    print('LastEpoch: ', model_checkpoint_epoch)

    model_epoch_info.update({subdir: {'best_valid_epoch': model['best_valid_epoch'], 'last_epoch': model_checkpoint_epoch}})
            

run_20231210_gelu_clean
Best Validation Loss Epoch:  17
LastEpoch:  30
run_20231206_relu_clean
Best Validation Loss Epoch:  28
LastEpoch:  30
run_20231211_selu_clean
Best Validation Loss Epoch:  6
LastEpoch:  30
run_20231208_singleswish_clean
Best Validation Loss Epoch:  6
LastEpoch:  30
run_20231209_tanh_clean
Best Validation Loss Epoch:  6
LastEpoch:  30
run_20231208_leakyRelu_clean
Best Validation Loss Epoch:  23
LastEpoch:  30
run_20231210_elu_clean
Best Validation Loss Epoch:  6
LastEpoch:  30
run_20231213_softplus_all_activation_clean
Best Validation Loss Epoch:  5
LastEpoch:  5
run_20231211_softplus_clean
Best Validation Loss Epoch:  26
LastEpoch:  30
run_20231206_doubleswish_clean
Best Validation Loss Epoch:  23
LastEpoch:  30


In [31]:
model['env_info']

{'k2-version': '1.24.4',
 'k2-build-type': 'Release',
 'k2-with-cuda': True,
 'k2-git-sha1': '59aef5e3a52a43990f78ad33f4839f0a1680358a',
 'k2-git-date': 'Thu Nov 23 09:08:57 2023',
 'lhotse-version': '1.17.0.dev+git.b869488.clean',
 'torch-version': '2.1.0+cu121',
 'torch-cuda-available': True,
 'torch-cuda-version': '12.1',
 'python-version': '3.1',
 'icefall-git-branch': 'master',
 'icefall-git-sha1': '7c682ec-dirty',
 'icefall-git-date': 'Sat Dec 9 06:50:50 2023',
 'icefall-path': '/home/ipsoct4/project/icefall_w4995_mathml',
 'k2-path': '/home/ipsoct4/anaconda3/envs/kaldi/lib/python3.11/site-packages/k2/__init__.py',
 'lhotse-path': '/home/ipsoct4/anaconda3/envs/kaldi/lib/python3.11/site-packages/lhotse/__init__.py',
 'hostname': 'instance-4t4',
 'IP address': '10.182.0.2'}

### 1.2 Run decoders and conduct evaluation

Run all experiements `run_all_decoders.sh` or 
individual experiment
`python decode.py --epoch 30 --avg 1 --max-duration 150  --exp-dir "$PWD/$dir_name"  --lang-dir ../data/lang_bpe_500  --method ctc-decoding`

Note you need to run individual experiement and change epoch argument manually to the best validation loss epoch obtained from step 1.1, `run_all_decoders.sh` does not support a variable epoch argument

### 1.3 Load experiments and obtain the WER

In [76]:
directory='./exp/'

wer_results = {}

for subdir in os.listdir(directory):
    wer_results.update({subdir: []})
    for file in os.listdir(directory + subdir):
        if re.match(r'wer.*clean\.txt$', file):
            with open(os.path.join(directory, subdir, file), 'r') as f:
                clean_wer_text = f.readlines()
                # extract number from string
                clean_wer = float(clean_wer_text[1].split('\t')[-1])
        elif re.match(r'wer.*other\.txt$', file):
            with open(os.path.join(directory, subdir, file), 'r') as f:
                other_wer_text = f.readlines()
                # extract number from string
                other_wer = float(other_wer_text[1].split('\t')[-1])
    wer_results.update({subdir: {'clean_wer': clean_wer, 'other_wer': other_wer}})

#### Word Error Rate Results

In [78]:
# NOTE: the wer results shown here are mix of last epoch (30) and best validation epoch (from step 1.1)
# as the author run some experiments individually
# Please refer to the paper for the correct wer results, separated by model using best validation epoch and last epoch
wer_results

{'run_20231210_gelu_clean': {'clean_wer': 9.93, 'other_wer': 25.42},
 'run_20231206_relu_clean': {'clean_wer': 12.72, 'other_wer': 33.96},
 'run_20231211_selu_clean': {'clean_wer': 97.73, 'other_wer': 97.74},
 'run_20231208_singleswish_clean': {'clean_wer': 98.7, 'other_wer': 98.76},
 'run_20231209_tanh_clean': {'clean_wer': 98.92, 'other_wer': 98.8},
 'run_20231208_leakyRelu_clean': {'clean_wer': 21.62, 'other_wer': 47.37},
 'run_20231210_elu_clean': {'clean_wer': 99.99, 'other_wer': 99.99},
 'run_20231213_softplus_all_activation_clean': {'clean_wer': 99.99,
  'other_wer': 99.99},
 'run_20231211_softplus_clean': {'clean_wer': 8.82, 'other_wer': 23.68},
 'run_20231206_doubleswish_clean': {'clean_wer': 8.28, 'other_wer': 22.21}}

## 2. Analysis on Computation Time

In [ ]:
# find all directories with tensorboard files, within each directory, find the latest event file, and parse it to get the epoch durations

computing_time = {}

# def calculate_avg_epoch_duration(directory='./exp/'):
    # get the name of all subdirectories
    # for each subdirectory name, prune the name to get the experiment name

directory='./exp/'
for subdir in os.listdir(directory):
    exp_name = subdir[13:]
    # print(subdir)
    # get the latest event file
    event_file = max(glob.glob(directory+ subdir + '/tensorboard/events.out.tfevents.*'), key=os.path.getctime)
    # parse the event file to get the epoch durations
    epoch_times = {}

    for event in tf.compat.v1.train.summary_iterator(event_file):
        for value in event.summary.value:
            if value.tag == "train/epoch":
                epoch_number = int(value.simple_value)
                if epoch_number not in epoch_times:
                    epoch_times[epoch_number] = event.wall_time

    
    last_available_epoch = min(max(epoch_times.keys()), 30)

    if min(epoch_times.keys()) > 25:
        first_available_epoch = min(epoch_times.keys())
    elif min(epoch_times.keys()) <= 6:
        first_available_epoch = min(epoch_times.keys())
    else:
        first_available_epoch = last_available_epoch - 5


    average_duration_raw = (epoch_times[last_available_epoch] - epoch_times[first_available_epoch]) / (last_available_epoch - first_available_epoch) 

    average_duration_processed = str(datetime.timedelta(seconds=average_duration_raw))[:7]

    computing_time[exp_name] = average_duration_processed
    
    # return computing_time

# calculate_avg_epoch_duration(directory='./exp')


#### The average computation time per epoch

Calculated using wall time and averaged over the last 5 epoch (since some experiment was not continously run, the wall time was not recorded correctly in the middle)

In [124]:
# Print the average computing time for each experiment
computing_time

{'gelu_clean': '0:32:16',
 'relu_clean': '0:31:19',
 'selu_clean': '0:32:11',
 'singleswish_clean': '0:33:00',
 'tanh_clean': '0:32:02',
 'leakyRelu_clean': '0:32:13',
 'elu_clean': '0:32:03',
 'softplus_all_activation_clean': '0:32:37',
 'softplus_clean': '0:32:27',
 'doubleswish_clean': '0:33:35'}